In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
train_data = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
test_data = datasets.MNIST(root="data", download=True, train=False, transform=ToTensor())

loaders = {
    "train": DataLoader(train_data, batch_size = 100, shuffle=True),
    "test": DataLoader(test_data, batch_size = 100, shuffle=True)
}

In [4]:
# Image Classifier Neural Network
class CNN(nn.Module): 
    def __init__(self):
        super(CNN,self).__init__()

        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(10,20,kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    def forward(self, x): 
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x)

In [5]:
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_index, (data, target) in enumerate(loaders['train']):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output,target)
        loss.backward()
        optimizer.step()
        if (batch_index % 20 == 0):
            print(f"Train Epoch: {epoch} [{batch_index * len(data)}/{len(loaders['train'].dataset)} ({100. * batch_index / len(loaders['train']) :.0f}%)]\t{loss.item():.6f}")


def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output,target).item()
            pred = output.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(loaders['test'].dataset)
    print(f"\nTest set: Average Loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders['test'].dataset)} ({100. * correct / len(loaders['test'].dataset):.0f}%\n)")

In [6]:
for epoch in range(1,11):
    train(epoch)
    test()

C:\Users\User\AppData\Local\Temp\ipykernel_10620\2419947235.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	2.303962
Train Epoch: 1 [2000/60000 (3%)]	2.291366
Train Epoch: 1 [4000/60000 (7%)]	2.201898
Train Epoch: 1 [6000/60000 (10%)]	1.997455
Train Epoch: 1 [8000/60000 (13%)]	1.913538
Train Epoch: 1 [10000/60000 (17%)]	1.905120
Train Epoch: 1 [12000/60000 (20%)]	1.817547
Train Epoch: 1 [14000/60000 (23%)]	1.841432
Train Epoch: 1 [16000/60000 (27%)]	1.791811
Train Epoch: 1 [18000/60000 (30%)]	1.825194
Train Epoch: 1 [20000/60000 (33%)]	1.637165
Train Epoch: 1 [22000/60000 (37%)]	1.751660
Train Epoch: 1 [24000/60000 (40%)]	1.726043
Train Epoch: 1 [26000/60000 (43%)]	1.751826
Train Epoch: 1 [28000/60000 (47%)]	1.751261
Train Epoch: 1 [30000/60000 (50%)]	1.692982
Train Epoch: 1 [32000/60000 (53%)]	1.681353
Train Epoch: 1 [34000/60000 (57%)]	1.630377
Train Epoch: 1 [36000/60000 (60%)]	1.649765
Train Epoch: 1 [38000/60000 (63%)]	1.613632
Train Epoch: 1 [40000/60000 (67%)]	1.662209
Train Epoch: 1 [42000/60000 (70%)]	1.629769
Train Epoch: 1 [44000/60000 (73%)]	1.581254